
# Introduction

---
Up to this point, we’ve covered the foundational concepts of machine learning. You may have already experimented with various hyperparameters to optimize model performance. In this notebook, you’ll be introduced to several advanced techniques designed to further enhance your models.


This notebook consists the following parts:

- [A: Data retrieval ](#01)
- [B: Feature engineering](#02)
- [C: Model evaluation](#03)
- [D: Ensemble learning](#04)
- [E: Pipelines](#05)
- [F: Bring it all together](#06)
- [G: Bonus: ML OPS](#07)


---

### Learning Objectives
By the end of this two weeks you will be able to:
- Understand the fundamental concepts of ensemble learning.
- Use evaluation techniques to assess models performance.
- Enhance model performance by feature engineering. 
- Build pipelines for model development and preprocessing

---

### Instructions
- Ensure you fully understand the requirements and objectives of the assignment.
- Review the notebooks refered in the tasks
- If you need additional context or clarification, please check the provided videos or background literature.
- Work through each part of the assignment methodically, ensuring all tasks are completed.
- Update your repository with your new created work

### Additional Notes:
- Do not add datafiles to your repository. Repositories with datafiles will not be accepted
- Class solutions should be delivered in python files. Not in notebooks
- When AI tools are used, you must provide proper references and explanations for how they were utilized. Failure to do so will be considered as academic fraud
- The bonus assignment are not mandatory
- Use PEP8 

Good luck!

F.Feenstra

---


<a name='01'></a>
## Part A. Data retrieval

The dataset you can use for this notebook is the lung dataset from Maastricht University. It comprises 89 non-small cell lung cancer (NSCLC) patients records who underwent surgical treatment. The study where the data is from explored the relationship between radiomic imaging features and gene expression profiles. The samples were collected through biopsies at MAASTRO Clinic in The Netherlands, and the dataset is publicly available.

The authors of the related paper discovered that a prognostic radiomic signature, which captures intra-tumor heterogeneity, is closely associated with underlying gene expression patterns. Developing a machine learning model to predict histology from the Clinical and Genetic Lung data can improve diagnostic accuracy and treatment personalization. In this notebook we will develop such a prediction model. 

**Availabel Datasets**:
- Lung metadata dataset [1]
- Gene expression dataset [2]

**Important**
<span style="background-color: lightgreen;color: black">It is also allowed to use your own dataset from your own project if this data is highly dimensional and contains genetic information.</span>

[1] [NSCLC-Radiomics-Genomics](https://wiki.cancerimagingarchive.net/display/Public/NSCLC-Radiomics-Genomics#16056856db10d39adf704eefa53e41edcf5ef41c)

[2] [Gene Expression Data - GSE58661](https://ftp.ncbi.nlm.nih.gov/geo/series/GSE58nnn/GSE58661/matrix/)

[3] Aerts HJWL, Rios Velazquez E, Leijenaar RTH, Parmar C, Grossmann P, Carvalho S, Bussink J, Monshouwer R, Haibe-Kains B, Rietveld D, Hoebers F, Rietbergen MM, Leemans CR, Dekker A, Quackenbush J, Gillies RJ, & Lambin P. (2015). Data From NSCLC-Radiomics-Genomics. The Cancer Imaging Archive. https://doi.org/10.7937/K9/TCIA.2015.L4FRET6Z


### <span style="background-color: lightyellow;">Data retrieval task</span>
- Retrieve the data. (No cleaning needed yet)

In [1]:
import pandas as pd
import gzip

lung_pd = pd.read_excel('Lung3.metadata.xls')
print(lung_pd.columns)
lung_pd.rename(columns=lambda x: x.replace('characteristics.tag.', ''), inplace=True)
print(lung_pd.columns)
cols = ['title', 'histology', 'tumor.size.maximumdiameter', 'stage.primary.tumor', 'gender']
lung_pd = lung_pd[cols]
lung_pd.dropna(inplace=True)
print(lung_pd.columns)
print(lung_pd.head())

lung_pd['histology'] = lung_pd['histology'].apply(lambda x: x.replace('\xa0', ' ').lower().strip())
uniques = set(list(lung_pd['histology']))


data_gse = pd.read_csv('GSE58661_series_matrix.txt.gz', sep='\t', comment='!', quotechar='"', compression='gzip')
data_gse.dropna(inplace=True)
data_gse.set_index("ID_REF", inplace=True)
print(data_gse.columns)
data_gse.head()

Index(['sample.name', 'title', 'CEL.file', 'source.location', 'organism',
       'characteristics.tag.gender', 'characteristics.tag.histology',
       'characteristics.tag.tumor.size.maximumdiameter',
       'characteristics.tag.stage.primary.tumor',
       'characteristics.tag.stage.nodes', 'characteristics.tag.stage.mets',
       'characteristics.tag.primaryVSmets', 'characteristics.tag.grade',
       'molecule tested', 'label', 'platform'],
      dtype='object')
Index(['sample.name', 'title', 'CEL.file', 'source.location', 'organism',
       'gender', 'histology', 'tumor.size.maximumdiameter',
       'stage.primary.tumor', 'stage.nodes', 'stage.mets', 'primaryVSmets',
       'grade', 'molecule tested', 'label', 'platform'],
      dtype='object')
Index(['title', 'histology', 'tumor.size.maximumdiameter',
       'stage.primary.tumor', 'gender'],
      dtype='object')
    title                       histology  tumor.size.maximumdiameter  \
0  lung_1    Squamous Cell Carcinoma, NOS     

,GSM1416528,GSM1416529,GSM1416530,GSM1416531,GSM1416532,GSM1416533,GSM1416534,GSM1416535,GSM1416536,GSM1416537,...,GSM1416607,GSM1416608,GSM1416609,GSM1416610,GSM1416611,GSM1416612,GSM1416613,GSM1416614,GSM1416615,GSM1416616
ID_REF,,,,,,,,,,,,,,,,,,,,,
AFFX-BioB-3_at,7.376915,8.024915,7.522543,7.152864,7.211031,7.704962,7.779374,7.136893,7.160221,8.141578,...,7.611992,7.345206,7.316975,7.418340,7.756286,7.215856,7.465109,7.245458,7.039592,7.393667
AFFX-BioB-5_at,6.984530,7.427048,7.077207,6.849513,6.753131,7.195947,7.429800,6.745630,6.751358,7.632631,...,7.175015,7.014479,6.913879,6.898449,7.347570,6.740992,7.080787,6.725812,6.575376,7.041106
AFFX-BioB-M_at,7.330576,8.010530,7.334551,7.143286,7.077163,7.753484,7.712429,7.036303,7.089186,7.992361,...,7.612393,7.285821,7.324635,7.408880,7.745624,7.152722,7.422373,7.263596,6.975162,7.366620
AFFX-BioC-3_at,10.922741,11.390638,10.936703,10.791909,10.701328,11.221732,11.084229,10.588372,10.628909,11.412668,...,11.097486,10.908682,10.781228,10.935723,11.200635,10.707980,10.736531,10.770440,10.367009,10.753221
AFFX-BioC-5_at,11.032030,11.533338,11.018510,10.875259,10.823792,11.276973,11.178643,10.700420,10.779332,11.470336,...,11.265739,10.990373,10.900765,11.101546,11.343329,10.842697,10.900509,10.838008,10.494806,10.863600


---
<a name='02'></a>
## Part B. Feature engineering

Mind you that choosing an algorithm and hyperparameter tuning might not be enough. If your data is of low quality, the algorithm will have poor performance as well. This is where feature engineering comes into play. Feature engineering involves transforming raw data into meaningful features that better represent the underlying problem to the predictive models, ultimately enhancing the model's performance. By selecting, creating, and refining features, you ensure that your data highlights the most relevant patterns and relationships, allowing the algorithm to learn more effectively. Proper feature engineering can often make the difference between a moderate model and a highly accurate one, even more so than the choice of algorithm itself. 
Possible modifications:
- creation of new features derived from original features
- selection of features
- encoding features
- log transformation 
- scaling
- dimension reduction (*e.g.* PCA)

### <span style="background-color: lightyellow;">Feature engineering Task</span>
- Review the [Study Case Feature Engineering notebook](../Study_Cases/study_case_feature_engineering.ipynb).
- Review the [Study Case RNA-seq Preparation notebook](../Study_Cases/study_case_scanpy_object.ipynb).
- Assess which data preparation and feature engineering steps could be beneficial for your dataset.
- Implement a `DataProcessor` class tailored to your dataset and test it in the cell below. **Make sure it's an appropriate sklearn object, such as a `Transformer` or `Pipeline`.**
- Update your repository with a new directory named `optimization`, including the following:
    - The `DataProcessor` class as a Python file, complete with thorough documentation.
    - An evaluation document that details and justifies your choices using a well-reasoned, argumentative approach.



In [2]:
# YOUR CODE HERE TO DEMONSTRATE THE USAGE OF THE PREPROCESSOR CLASS
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

data_gse_trans = data_gse.transpose()

# To combat skewed data 
data_gse_log = np.log1p(data_gse_trans)

#Standardizeing since genes can have very different scales of expression.
scaler = StandardScaler()
scaled_gse_trans_log = scaler.fit_transform(data_gse_log)

# Performing PCA I know there is a better way to do like doing a pca on all data and then look at what point it becomes more than 0.5 total. 
# But this is to practice recursion.
def find_components(data, start=2):
    pca = PCA(n_components=start)
    pca_res = pca.fit_transform(data)
    expl_var_list = pca.explained_variance_ratio_

    if sum(expl_var_list) >= 0.5:
        print(f"N component to reach 50% explained of the variance: {start}")
        print(f"Explained variance list: {expl_var_list}")
        return start, pca, expl_var_list, pca_res
    else:
        start += 1
        return find_components(data, start)


start, pca, expl_var_list, pca_res = find_components(scaled_gse_trans_log)
columns1 = [f"PCA_{i+1}" for i in range(start)]


pca_res_df = pd.DataFrame(data=pca_res, columns=columns1) 

merged_df = pd.merge(lung_pd, pca_res_df, left_index=True, right_index=True)
print(merged_df.head())

N component to reach 50% explained of the variance: 16
Explained variance list: [0.09358163 0.08701857 0.04763505 0.04399918 0.03330774 0.02687047
 0.02465432 0.02241215 0.01850155 0.01716716 0.01618323 0.01520436
 0.01432665 0.01350128 0.01334884 0.01303402]
    title                       histology  tumor.size.maximumdiameter  \
0  lung_1    squamous cell carcinoma, nos                         4.0   
1  lung_2  adenocarcinoma, papillary, nos                         1.3   
2  lung_3                  non-small cell                        11.0   
4  lung_5    squamous cell carcinoma, nos                         7.8   
5  lung_6             adenocarcinoma, nos                         3.5   

  stage.primary.tumor gender      PCA_1       PCA_2      PCA_3      PCA_4  \
0                 pT2      M -12.563557  143.351731 -74.116869 -29.403301   
1                 pT1      M -14.590991 -136.112014 -31.727267 -93.328784   
2                 pT3      M -72.357969   31.079279  -7.229756 -23.025

In [3]:
# Encoder used for turning strings/classifications into numeric values
from sklearn.preprocessing import LabelEncoder

def encode_column(df, column_enc):
    encoder = LabelEncoder()
    df[column_enc] = encoder.fit_transform(df[column_enc])
    return df

merged_df = encode_column(merged_df, 'histology')
merged_df = encode_column(merged_df, 'stage.primary.tumor')

In [1]:
from DataProcessor import DataProcessor

pipeline = DataProcessor(input_file_exp = 'GSE58661_series_matrix.txt.gz', input_file_meta = "Lung3.metadata.xls",
                         cols_to_grab = ['title',
       'characteristics.tag.gender', 'characteristics.tag.histology',
       'characteristics.tag.tumor.size.maximumdiameter',
       'characteristics.tag.stage.primary.tumor'])

pipeline_finished, encoders = pipeline.pipeline_run()

print(pipeline_finished.head())
pipeline.look_at_encode(encoders, "title")




N component to reach 50% explained of the variance: 14
Explained variance list: [0.13625197 0.0657839  0.04725546 0.04508164 0.03637917 0.02773846
 0.02481676 0.02130975 0.02095438 0.01704936 0.01577596 0.0155997
 0.01448788 0.01410956]
['title', 'gender', 'histology', 'stage.primary.tumor']
    title gender                       histology  tumor.size.maximumdiameter  \
0  lung_1      M    Squamous Cell Carcinoma, NOS                         4.0   
1  lung_2      M  Adenocarcinoma, Papillary, NOS                         1.3   
2  lung_3      M                  Non-Small Cell                        11.0   
4  lung_5      F    Squamous Cell Carcinoma, NOS                         7.8   
5  lung_6      M             Adenocarcinoma, NOS                         3.5   

  stage.primary.tumor      PCA_1     PCA_2     PCA_3      PCA_4     PCA_5  \
0                 pT2  15.075251 -5.472487  0.142375   2.999663  3.193272   
1                 pT1  -6.797374  9.108923 -4.152204  10.089548 -3.86586

LabelEncoder()

<a name='03'></a>
# Part C. Model Evaluation

Evaluating the performance of a machine learning model goes beyond just looking at accuracy, as accuracy alone can be misleading, especially in cases where the dataset is imbalanced or where different types of errors have different consequences
- **Detecting Overfitting/Underfitting**: A learning curve can help you understand whether your model is overfitting (performing well on training data but poorly on validation data) or underfitting (performing poorly on both training and validation data)
- **ROC**: The ROC curve shows how well your model distinguishes between classes. It helps in selecting the optimal threshold for classification decisions, particularly when the cost of false positives and false negatives differs significantly. ROC curves are often used to compare models. 
- **Confusion matrix**: From the confusion matrix, you can derive other important metrics like precision, recall, F1-score, and specificity, which give a better understanding of how your model is performing across different classes


### <span style="background-color: lightyellow;">Evaluation Task</span>
- Review the documentation for `sklearn.metrics` and `learning_curve`.
- Select appropriate metrics for your dataset, including accuracy and indicators of overfitting or underfitting.
- Implement functions to compute and assess these metrics. It is allowed to use libraries.
- Add the evaluation functions as a Python module to your repository.
- Update the evaluation documentation to clearly explain your choices for the evaluation metrics and the rationale behind them.

See also: [Model evaluation video](https://video.hanze.nl/media/model-evaluation/0_gybpnhq7)


In [ ]:
# YOUR CODE HERE TO DEMONSTRATE THE USAGE OF THE EVALUATION MODULE

<a name='04'></a>
## Part D. Ensemble learning
Ensemble learning is a powerful machine learning technique that combines the predictions of multiple models to improve overall performance, robustness, and accuracy. Rather than relying on a single model, ensemble learning methods aggregate the results of several models—often called "weak learners"—to produce a stronger predictive model. The key idea behind ensemble learning is that by combining models, the weaknesses of individual models can be offset, leading to better generalization on unseen data. Popular ensemble ML algorithms are the `Random Forest` and `XGBoost`. 
Here’s an improved version of the introduction:

### voting algorithms
Voting algorithms in ensemble learning combine the predictions of multiple classifiers to make a final decision, typically based on the consensus or weighted agreement among models. Two primary types of voting are commonly used: hard voting, where the final prediction is determined by the majority vote, and soft voting, which uses the weighted average of predicted probabilities to determine the outcome. Several sites explain the hard and soft voting algorithm. A clear explanation can be found on https://www.baeldung.com/cs/hard-vs-soft-voting-classifiers


### <span style="background-color: lightyellow;">Ensemble Task</span>
- Review the [Study Case notebook on ensemble learning](..Study_Cases/study_case_bagging_boosting.ipynb).
- Try three different algorithms for classification of your data label.
- Implement a hard and soft voting algorithm for model aggregation.
- Compare the performance of the voting algorithm with that of a boosting or bagging algorithm.
- Update your repository with the voting algorithm class

See also: [ensemle learning video](https://video.hanze.nl/media/Ensemble/0_sue5v33g)


In [ ]:
# YOUR CODE HERE WITH THE VOTING ALGORITHMS
def vote_soft():
    pass

def vote_hard():
    pass

In [ ]:
# YOUR CODE HERE TO COMPARE VOTING OUTCOME WITH A BAGGING OR BOOSTING ALGORITHM FROM SKLEARN

<a name='05'></a>
## Part E. Pipelines
The sklearn pipeline function is a tool in machine learning that simplifies the workflow by encapsulating all the steps involved in a single object. It offers advantages such as simplicity, reproducibility, efficiency, flexibility, and integration. The Pipeline is built using a list of (key, value) pairs, where the key is a string containing the name you want to give this step and value is an estimator object (the method to be executed).

### <span style="background-color: lightyellow;">Pipeline Task</span>
- Read the [Study Case notebook for a pipeline functions](..Study_Cases/study_case_pipeline.ipynb) to understand the principle of the pipeline function
- Implement a pipeline which prepares and classifies data 
- Use a `GridSearchCV` object with the `Pipeline` object and a parameter grid to optimize choose the best hyper parameters

In [ ]:
# YOUR CODE HERE WITH THE PIPELINE IMPLEMENTATION

## <span style="background-color: lightyellow;">Bring it all together</span>

By now, you've developed code snippets for model evaluation, optimization, and data improvement. Now, leverage these skills to build a classification model using the Clinical and Genetic Lung data. Make sure that you log your experiments. 

Once you're satisfied with the model, upload the relevant code to your repository and or refactor code with new insights. Furthermore, take a moment to reflect on its applicability and potential real-world impact. Update your evaluation document(s) with these findings in your repository. 

## Bonus: ML for operations

If we intend to deploy the model in a real-world application, it's more efficient to save and reuse the trained model rather than retraining it each time. 

- Read the blog: https://neptune.ai/blog/saving-trained-model-in-python
- Write three python files 
    1) a train_model python file 
    2) a use_model python file
    3) a retrain_model python file that adds new data to the original training data and updates the model
- Update your repository
